In [23]:
import pandas as pd
import numpy as np
import random as rd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")


### 1: EMPLOYEE MASTER DATA

employees = 20000
active_employees = 750 
df = pd.DataFrame()                                                              

df["employee_id"] = df.index + 1
df["corporation"] = np.random.choice(["corporate","sport"], employees, p=[0.4, 0.6])
df["gender"] = np.random.choice(["male","female"], employees, p=[0.7, 0.3])
df["age"] = np.random.choice(["under 25","25-35","35-50","50-66","over 66"], employees, p=[0.18, 0.31, 0.27, 0.15, 0.09])

def age_start (age_str):
    if age_str == "over 66":
        return age_str[-2:]
    elif age_str == "under 25":
        return "18"
    else:
        return age_str[:2]
    
def age_end (age_str):
    if age_str == "over 66":
        return "80"
    else:
        return age_str[-2:]

df["age_interval_start"] = df["age"].apply(age_start).astype(int)
df["age_interval_end"] = df["age"].apply(age_end).astype(int)                                                            
df["educational_level"] = np.random.choice(["High School", "Bachelor's Degree", "Master's Degree", "PhD"], employees, 
                                           p=[0.3, 0.4, 0.2, 0.1])
df["management_level"] = np.random.choice(["top", "middle", "first line", "support", "specialized technicians"], 
                                          employees, p=[0.05, 0.1, 0.18, 0.61, 0.06])
df["type_of_contract"] = np.random.choice(["full time", "part time", "intern", "freelancer"], employees, 
                                          p=[0.7, 0.2, 0.05, 0.05])

df["working_service"] = np.where((df["type_of_contract"] == "full time") | 
                                 (df["type_of_contract"] == "part time") | 
                                 (df["type_of_contract"] == "intern"), 
                                 "fixed contract","occasional work")
df["department"] = np.where(df["corporation"] == "corporate", np.random.choice(["HR", "Marketing & Sales", 
                            "Controlling", "Finance & Administration", "IT", "Media & Contents"], employees),
                            np.random.choice(["Scouting", "Youth Football", "Professional Football"], employees))

def generate_random_tenure(age_interval_end):
    upper_bound = (age_interval_end*12)-(18*12)
    return rd.randint(1, upper_bound)

df['tenure_in_months'] = df['age_interval_end'].apply(generate_random_tenure)
df["tenure_in_years"] = (df["tenure_in_months"] / 12).astype(int)

df['active_employee'] = 0  
active_indices = rd.sample(range(len(df)), active_employees)
df.loc[active_indices, "active_employee"] = 1

df['terminated_by_employee'] = np.where(df['active_employee'] == 0, np.random.choice([1, 0], employees, p=[0.85, 0.15]), None)
df["termination_reason"] = np.where(df["terminated_by_employee"] == 1, 
                                    np.random.choice(["New Professional Challenge", "Salary Package", 
                                                     "Lack of Growth Perspective", "Work Schedule",
                                                     "Family Reasons", "Contractual Bond", "Workplace", 
                                                     "Conflict with Management", "Conflict with Colleagues",
                                                     "Very Procedural and Repetitive Functions", "Personal Reasons",
                                                     "Expectations not Met", "Search for Professional Development",
                                                     "Return to Studies"], employees, 
                                                     p=[0.23, 0.17, 0.15, 0.05, 0.08, 0.02, 0.03, 0.01, 0.02, 0.01,
                                                        0.16, 0.02, 0.04, 0.01]), None)

def calculate_start_date(row):
    tenure_in_months = row['tenure_in_months']
    active_employee = row['active_employee']   
    if active_employee == 1:
        start_date = datetime.now() - timedelta(days=tenure_in_months * 30)  # Approximating 30 days per month
    else:
        start_date_min = datetime(1963, 1, 1)
        start_date_max = datetime.now() - timedelta(days=tenure_in_months * 30)
        if start_date_min < start_date_max:
            start_date = start_date_min + timedelta(days=rd.randint(0, (start_date_max - start_date_min).days))
        else:
            start_date = start_date_min
    return start_date.date()

df['start'] = df.apply(calculate_start_date, axis=1)
df["end"] = df.apply(lambda row: pd.to_datetime(row["start"]) + timedelta(days=row["tenure_in_years"] * 365) 
                          if row["active_employee"] == 0 else None, axis=1)

df["start"] = df["start"].apply(lambda x: x.year)
df["end"] = df["end"].apply(lambda x: x.year)
df['end'] = df['end'].fillna(0).astype(int)

df["benefit_healthcare"] = np.random.choice([1, 0], employees)
df["benefit_flexible_work"] = np.random.choice([1, 0], employees)
df["benefit_gym_membership"] = np.random.choice([1, 0], employees)
df["benefit_navegante"] = np.random.choice([1, 0], employees)
df["benefit_benfica_membership"] = np.random.choice([1, 0], employees)

def score(row):
    if row['end'] == 2023:
        return np.random.choice([1, 2, 3, 4, 5], p=[0.4, 0.35, 0.1, 0.1, 0.05])
    else:
        return np.random.choice([1, 2, 3, 4, 5], p=[0.05, 0.1, 0.25, 0.3, 0.3])

df["score_work_life_balance"] = df.apply(score, axis=1)
df["score_colleagues"] = df.apply(score, axis=1)
df["score_compensation"] = df.apply(score, axis=1)
df["score_career_opportunities"] = df.apply(score, axis=1)
df["score_job_tasks"] = df.apply(score, axis=1)

df["employee_id"] = df.index + 1


### 2: HISTORICAL EMPLOYEE DATA

df_hist = df
duplicated_rows = []

for _, row in df_hist.iterrows():
        duplicated_rows.extend([row] * (row["tenure_in_years"] + 1))

df_hist = pd.DataFrame(duplicated_rows, columns=df_hist.columns).reset_index(drop=True)

df_hist["year"] = df_hist["start"]

def increment_years(group):
    group["year"] += np.arange(len(group))
    return group

df_hist = df_hist.groupby("employee_id").apply(increment_years)
df_hist = df_hist.reset_index(drop=True)

df_hist = df_hist[df_hist.year >= 2005]

df_hist = df_hist.loc[df_hist.index.repeat(12)].reset_index(drop=True)
df_hist.reset_index(drop=True, inplace=True)

df_hist['month_number'] = df_hist.groupby(['employee_id', 'year']).cumcount() % 12 + 1

def mon(row):
    if row["month_number"] == 1:
        return "Jan"
    if row["month_number"] == 2:
        return "Feb"
    if row["month_number"] == 3:
        return "Mar"
    if row["month_number"] == 4:
        return "Apr"
    if row["month_number"] == 5:
        return "May"
    if row["month_number"] == 6:
        return "Jun"
    if row["month_number"] == 7:
        return "Jul"
    if row["month_number"] == 8:
        return "Aug"
    if row["month_number"] == 9:
        return "Sep"
    if row["month_number"] == 10:
        return "Oct"
    if row["month_number"] == 11:
        return "Nov"
    if row["month_number"] == 12:
        return "Dec"
    
df_hist["month"] = df_hist.apply(mon, axis=1)

inactive_rows = df_hist[df_hist['active_employee'] == 0].copy()

inactive_rows.sort_values(['employee_id', 'year', 'month_number'], ascending=[True, False, False], inplace=True)

inactive_rows['drop'] = 0  # Initialize the drop column
for _, group in inactive_rows.groupby('employee_id'):
    num_ones = rd.randint(0, 11)
    if num_ones > 0:
        last_rows = group.head(num_ones).index
        inactive_rows.loc[last_rows, 'drop'] = 1

df_hist['drop'] = np.where(df_hist['active_employee'] == 1, 0, inactive_rows['drop'].reindex(df_hist.index, fill_value=0))

df_hist = df_hist[df_hist["drop"] == 0]

inactive_rows = df_hist[df_hist['start'] == df_hist['year']].copy()
inactive_rows.sort_values(['employee_id', 'year', 'month_number'], ascending=[True, True, True], inplace=True)
inactive_rows['drop'] = 0  # Initialize the drop column

for _, group in inactive_rows.groupby('employee_id'):
    num_ones = rd.randint(0, 11)
    if num_ones > 0:
        last_rows = group.head(num_ones).index
        inactive_rows.loc[last_rows, 'drop'] = 1

df_hist['drop'] = np.where(df_hist['active_employee'] == 1, 0, inactive_rows['drop'].reindex(df_hist.index, fill_value=0))

df_hist = df_hist[df_hist["drop"] == 0]
df_hist = df_hist.drop(columns="drop")

salary_dict = {}

for index, row in df_hist.iterrows():
    employee_id = row["employee_id"]
    level = row["management_level"]
    if level == "top":
        salary_value = rd.randint(4200, 6000)
    elif level == "middle":
        salary_value = rd.randint(2900, 4200)
    elif level == "first line":
        salary_value = rd.randint(2300, 3300)
    else:
        salary_value = rd.randint(1600, 2300)
    salary_dict[employee_id] = salary_value

df_hist["base_salary"] = df_hist["employee_id"].map(salary_dict)

def benefits(row):
    x = 0
    if row["benefit_healthcare"] == 1:
        x += 120
    if row["benefit_flexible_work"] == 1:
        x += 50
    if row["benefit_gym_membership"] == 1:
        x += 35
    if row["benefit_navegante"] == 1:
        x += 40
    if row["benefit_benfica_membership"] == 1:
        x += 35
    return x

df_hist["monthly_benefits"] = df_hist.apply(benefits, axis=1)
df_hist["salary_plus_benefits"] = df_hist["base_salary"] + df_hist["monthly_benefits"]

df_hist["promotion"] = 0  
promotions_counter = {}

for index, row in df_hist.iterrows():
    employee_id = row["employee_id"]    
    if promotions_counter.get(employee_id, 0) < (row["tenure_in_years"] / 4):
        promotion_value = np.random.choice([0, 1], p=[0.7, 0.3])
        df_hist.at[index, 'promotion'] = promotion_value
        promotions_counter[employee_id] = promotions_counter.get(employee_id, 0) + promotion_value

df_hist['promotion'] = df_hist['promotion'].astype(int)

def calculate_salary_increase(row):
    if row['promotion'] == 1:
        increase_percentage = np.random.uniform(0.01, 0.10)
        return row['base_salary'] * increase_percentage
    else:
        return 0

df_hist['salary_increase'] = df_hist.apply(calculate_salary_increase, axis=1)
df_hist['salary_increase'] = df_hist['salary_increase'].astype(int)
df_hist["salary_increase_rel"] = round(df_hist["salary_increase"] / df_hist["salary_plus_benefits"], 2)
df_hist['full_salary'] = df_hist.groupby('employee_id')['salary_increase'].cumsum() + df_hist['salary_plus_benefits']

def reg_hours(emp):
    if (emp == "full time") | (emp == "intern"):
        return 4*40
    if emp == "part time":
        return 4*20
    if emp == "freelancer":
        return rd.randrange(8, 4*40, 8)
    
df_hist["regular_hours"] = df_hist["type_of_contract"].apply(reg_hours)

df_hist["vacation_hours"] = (df_hist["regular_hours"] * (30*8)/(12*4*40)).astype(int)

def sick_hours(row):
    max_hours = int((row["regular_hours"] - row["vacation_hours"])*0.3)
    return rd.randint(0, max_hours)

df_hist["sickness_hours"] = df_hist.apply(sick_hours, axis=1)
df_hist["absenteeism_in_days"] = (df_hist["vacation_hours"] + df_hist["sickness_hours"]) / 8

def over(reg):
    return rd.randint(0, int(reg*0.18))

df_hist["overtime_hours"] = df_hist["regular_hours"].apply(over)

df_hist = df_hist.drop(columns=["corporation", "gender", "age", "age_interval_start",
                                "age_interval_end", "educational_level", "type_of_contract", 
                                "working_service", "department", "management_level", 
                                "tenure_in_months", "tenure_in_years", "terminated_by_employee", 
                                "termination_reason", "benefit_healthcare", "benefit_flexible_work", 
                                "benefit_gym_membership", "benefit_navegante", "benefit_benfica_membership"])

df_hist = df_hist[df_hist.year <= 2023]
df_hist = df_hist[df_hist.end <= 2023]

df_hist = df_hist.sort_values(by=['employee_id', 'year', 'month_number'])
df_hist['starter'] = df_hist.groupby('employee_id').cumcount() == 0
df_hist['starter'] = df_hist['starter'].astype(int)
df_hist.reset_index(drop=True, inplace=True)

df_hist = df_hist.sort_values(by=['employee_id', 'year', 'month_number'])
df_hist['leaver'] = ((df_hist.groupby('employee_id')['active_employee'].transform('last') == 0) 
                     & (df_hist.groupby('employee_id').cumcount(ascending=False) == 0))
df_hist["leaver"] = df_hist['leaver'].astype(int)
df_hist.reset_index(drop=True, inplace=True)


### 3: APPLICANT DATA

applicants = 50000
hired = 20000

employee_ids = np.random.permutation(range(1, hired + 1))
df_app = pd.DataFrame({'employee_id': employee_ids[:hired]})
df_app = df_app.append(pd.DataFrame({'employee_id': [0] * (applicants - hired)}), ignore_index=True)

df_app = pd.merge(df_app, df[["employee_id", "start", "gender", "age", "department", "corporation", "educational_level", 
                              "type_of_contract", "management_level"]], on='employee_id', how='left')

def start(row):
    if row["employee_id"] == 0:
        return rd.randint(1963,2023)
    else:
        return int(row["start"])

df_app["start"] = df_app.apply(start, axis=1)

def gender(row):
    if row["employee_id"] == 0:
        return np.random.choice(["male", "female"], p=[0.7, 0.3])
    else: 
        return row["gender"]

df_app["gender"] = df_app.apply(gender, axis=1)

def age(row):
    if row["employee_id"] == 0:
        return np.random.choice(["under 25", "25-35", "35-50", "50-66", "over 66"])
    else: 
        return row["age"]

df_app["age"] = df_app.apply(age, axis=1)
    
def corp(row):
    if row["employee_id"] == 0:
        return np.random.choice(["corporate", "sport"], p=[0.4, 0.6])
    else: 
        return row["corporation"]

df_app["corporation"] = df_app.apply(corp, axis=1)
    
def dep(row):
    if (row["employee_id"] == 0) & (row["corporation"] == "corporate"):
        return np.random.choice(["HR", "Marketing & Sales", "Controlling", "Finance & Administration", "IT", 
                                 "Media & Contents"])
    elif (row["employee_id"] == 0) & (row["corporation"] == "sport"):
        return np.random.choice(["Scouting", "Youth Football", "Professional Football"])
    else:
        return row["department"]

df_app["department"] = df_app.apply(dep, axis=1)

def edu(row):
    if row["employee_id"] == 0:
        return np.random.choice(["High School", "Bachelor's Degree", "Master's Degree", "PhD"], p=[0.3, 0.4, 0.2, 0.1])
    else: 
        return row["educational_level"]

df_app["educational_level"] = df_app.apply(edu, axis=1)

def con(row):
    if row["employee_id"] == 0:
        return np.random.choice(["full time", "part time", "intern", "freelancer"], p=[0.7, 0.2, 0.05, 0.05])
    else: 
        return row["type_of_contract"]

df_app["type_of_contract"] = df_app.apply(con, axis=1)

def man(row):
    if row["employee_id"] == 0:
        return np.random.choice(["top", "middle", "first line", "support", "specialized technicians"], 
                                p=[0.02, 0.64, 0.1, 0.18, 0.06])
    else: 
        return row["management_level"]
    
df_app["management_level"] = df_app.apply(man, axis=1)

df_app["source"] = np.random.choice(["LinkedIn", "SLB Employee", "SLB Website", "Job Center", "Friends & Family", "Other"], 
                                    applicants, p=[0.35,0.04,0.19,0.08,0.21,0.13])

def step5(row):
    if row["employee_id"] != 0:
        return 1
    else:
        return 0

df_app["hired"] = df_app.apply(step5, axis=1)

def step4(row):
    if row["hired"] == 1:
        return 1
    else:
        return rd.randint(0,1)

df_app["trial_work"] = df_app.apply(step4, axis=1)

def step3(row):
    if row["trial_work"] == 1:
        return 1
    else:
        return rd.randint(0,1)

df_app["interview"] = df_app.apply(step3, axis=1)

def step2(row):
    if row["interview"] == 1:
        return 1
    else:
        return rd.randint(0,1)

df_app["screening"] = df_app.apply(step2, axis=1)

def offer(row):
    if (row["trial_work"] == 1) & (row["hired"] == 0):
        return np.random.choice([0,1], p=[0.73,0.27])
    else:
        return 0

df_app["offer_rejected"] = df_app.apply(offer, axis=1)

df_app["time_hired"] = df_app.apply(lambda x: np.random.randint(1, 17) if x["hired"] == 1 else 0, axis=1)
df_app["time_trial_work"] = df_app.apply(lambda x: np.random.randint(1, 19) if x["trial_work"] == 1 else 0, axis=1)
df_app["time_interview"] = df_app.apply(lambda x: np.random.randint(1, 20) if x["interview"] == 1 else 0, axis=1)
df_app["time_screening"] = df_app.apply(lambda x: np.random.randint(1, 26) if x["screening"] == 1 else 0, axis=1)
df_app["time_application"] = df_app.apply(lambda x: np.random.randint(7, 26), axis=1)

df_app["time_to_hire"] = df_app["time_screening"]+df_app["time_interview"]+df_app["time_trial_work"]+df_app["time_hired"]
df_app["time_to_fill"] = df_app["time_to_hire"]+df_app["time_application"]


df_app["cost_hired"] = df_app.apply(lambda x: 1200+x["time_hired"]*10 if x["hired"] == 1 else 0, axis=1)
df_app["cost_trial_work"] = df_app.apply(lambda x: 230+x["time_trial_work"]*10 if x["trial_work"] == 1 else 0, axis=1)
df_app["cost_interview"] = df_app.apply(lambda x: 160+x["time_interview"]*10 if x["interview"] == 1 else 0, axis=1)
df_app["cost_screening"] = df_app.apply(lambda x: 70+x["time_screening"]*10 if x["screening"] == 1 else 0, axis=1)

df_app["cost_to_hire"] = df_app["cost_screening"]+df_app["cost_interview"]+df_app["cost_trial_work"]+df_app["cost_hired"]

df.to_csv("benfica_data.csv")
df_hist.to_csv("emp_history.csv")
df_app.to_csv("emp_app.csv")


In [35]:
df

,employee_id,corporation,gender,age,age_interval_start,age_interval_end,educational_level,management_level,type_of_contract,working_service,...,benefit_healthcare,benefit_flexible_work,benefit_gym_membership,benefit_navegante,benefit_benfica_membership,score_work_life_balance,score_colleagues,score_compensation,score_career_opportunities,score_job_tasks
0,1,sport,male,35-50,35,50,Bachelor's Degree,support,full time,fixed contract,...,0,0,1,1,0,2,5,3,5,3
1,2,corporate,female,over 66,66,80,Master's Degree,support,full time,fixed contract,...,1,0,1,0,1,3,2,4,4,3
2,3,corporate,male,50-66,50,66,PhD,support,full time,fixed contract,...,0,0,1,0,1,4,3,3,3,3
3,4,corporate,male,25-35,25,35,High School,support,part time,fixed contract,...,1,0,0,0,1,2,4,4,5,4
4,5,sport,male,under 25,18,25,Bachelor's Degree,top,full time,fixed contract,...,0,1,1,1,1,5,4,4,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,sport,male,25-35,25,35,Bachelor's Degree,support,full time,fixed contract,...,0,0,0,0,1,5,2,5,3,4
19996,19997,sport,female,35-50,35,50,PhD,support,full time,fixed contract,...,1,1,0,0,0,3,3,1,5,1
19997,19998,sport,female,under 25,18,25,Master's Degree,middle,part time,fixed contract,...,0,1,0,0,1,5,5,5,5,4
19998,19999,sport,female,35-50,35,50,PhD,support,full time,fixed contract,...,1,0,0,0,1,5,2,1,1,3


In [39]:
df_hist

,employee_id,active_employee,start,end,year,month_number,month,base_salary,annual_benefits,salary_plus_benefits,...,salary_increase,salary_increase_rel,full_salary,regular_hours,vacation_hours,sickness_hours,absenteeism_in_days,overtime_hours,starter,leaver
0,1,0,1963,2023,2005,1,Jan,2070,90,2160,...,36,0.02,2196,160,20,7,3.375,13,1,0
1,1,0,1963,2023,2005,2,Feb,2070,90,2160,...,0,0.00,2196,160,20,35,6.875,22,0,0
2,1,0,1963,2023,2005,3,Mar,2070,90,2160,...,0,0.00,2196,160,20,39,7.375,27,0,0
3,1,0,1963,2023,2005,4,Apr,2070,90,2160,...,0,0.00,2196,160,20,33,6.625,27,0,0
4,1,0,1963,2023,2005,5,May,2070,90,2160,...,0,0.00,2196,160,20,21,5.125,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835466,19998,0,2020,2020,2020,3,Mar,2069,280,2349,...,0,0.00,2349,80,10,1,1.375,0,0,0
835467,19998,0,2020,2020,2020,4,Apr,2069,280,2349,...,0,0.00,2349,80,10,6,2.000,8,0,0
835468,19998,0,2020,2020,2020,5,May,2069,280,2349,...,0,0.00,2349,80,10,13,2.875,0,0,0
835469,19998,0,2020,2020,2020,6,Jun,2069,280,2349,...,0,0.00,2349,80,10,20,3.750,7,0,0


In [37]:
df_app

,employee_id,start,gender,age,department,corporation,educational_level,type_of_contract,management_level,source,...,time_interview,time_screening,time_application,time_to_hire,time_to_fill,cost_hired,cost_trial_work,cost_interview,cost_screening,cost_to_hire
0,1321,1991,male,35-50,Finance & Administration,corporate,High School,full time,support,Job Center,...,10,25,10,57,67,1360,290,260,320,2230
1,6051,1999,male,25-35,Scouting,sport,High School,full time,first line,LinkedIn,...,14,25,10,61,71,1320,330,300,320,2270
2,17035,1973,female,50-66,Scouting,sport,Master's Degree,part time,support,Friends & Family,...,1,14,18,35,53,1360,270,170,210,2010
3,3277,1974,male,35-50,Finance & Administration,corporate,High School,part time,first line,Friends & Family,...,15,5,25,44,69,1290,380,310,120,2100
4,938,1963,female,25-35,IT,corporate,Bachelor's Degree,full time,first line,LinkedIn,...,18,13,8,40,48,1280,240,340,200,2060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,1991,male,25-35,Youth Football,sport,Bachelor's Degree,part time,middle,SLB Website,...,5,18,11,30,41,0,300,210,250,760
49996,0,2021,male,50-66,Marketing & Sales,corporate,Bachelor's Degree,intern,first line,Other,...,0,19,8,19,27,0,0,0,260,260
49997,0,1986,male,under 25,Professional Football,sport,Bachelor's Degree,full time,support,SLB Website,...,11,10,15,23,38,0,250,270,170,690
49998,0,2021,female,25-35,HR,corporate,Bachelor's Degree,freelancer,middle,LinkedIn,...,15,13,22,37,59,0,320,310,200,830
